<a href="https://colab.research.google.com/github/elimeyer1/ML_4105/blob/main/Homework_7_Quesiton_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
num_epochs = 200
batch_size = 64
learning_rate = 0.001

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:02<00:00, 79.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [8]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

In [9]:
model = CNN().to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    training_time = time.time() - start_time
    return training_time, avg_loss

In [12]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [13]:
training_time, training_loss = train_model(model, train_loader, criterion, optimizer, num_epochs)
accuracy = evaluate_model(model, test_loader)

Epoch [1/200], Loss: 1.4345
Epoch [2/200], Loss: 1.0931
Epoch [3/200], Loss: 0.9440
Epoch [4/200], Loss: 0.8409
Epoch [5/200], Loss: 0.7603
Epoch [6/200], Loss: 0.6924
Epoch [7/200], Loss: 0.6264
Epoch [8/200], Loss: 0.5627
Epoch [9/200], Loss: 0.5026
Epoch [10/200], Loss: 0.4445
Epoch [11/200], Loss: 0.3989
Epoch [12/200], Loss: 0.3456
Epoch [13/200], Loss: 0.3036
Epoch [14/200], Loss: 0.2577
Epoch [15/200], Loss: 0.2225
Epoch [16/200], Loss: 0.1948
Epoch [17/200], Loss: 0.1607
Epoch [18/200], Loss: 0.1474
Epoch [19/200], Loss: 0.1276
Epoch [20/200], Loss: 0.1115
Epoch [21/200], Loss: 0.1041
Epoch [22/200], Loss: 0.0882
Epoch [23/200], Loss: 0.0952
Epoch [24/200], Loss: 0.0745
Epoch [25/200], Loss: 0.0719
Epoch [26/200], Loss: 0.0860
Epoch [27/200], Loss: 0.0653
Epoch [28/200], Loss: 0.0668
Epoch [29/200], Loss: 0.0662
Epoch [30/200], Loss: 0.0639
Epoch [31/200], Loss: 0.0592
Epoch [32/200], Loss: 0.0590
Epoch [33/200], Loss: 0.0519
Epoch [34/200], Loss: 0.0607
Epoch [35/200], Loss: 0

In [15]:
model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Model Size: {model_size} parameters')
print(f'Training Time: {training_time:.2f} seconds')
print(f'Training Loss: {training_loss:.4f}')
print(f'Test Accuracy: {accuracy:.2f}%')

Model Size: 268650 parameters
Training Time: 10706.91 seconds
Training Loss: 0.0296
Test Accuracy: 66.55%
